In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies = movies.merge(credits,on='title')

In [5]:
# Loop through each row in the DataFrame
import json
titles = []
overviews = []
keywords = []
productions = []
cast = []
characters = []
corpus2 = ""
def get_dict_values(dict_list):
    values = []
    for d in dict_list:
        if 'name' in d:  # Check if 'name' key exists in the dictionary
            values.append(d['name'])
    return values
def get_charchter_values(dict_list):
    values = []
    for d in dict_list:
        if 'character' in d:  # Check if 'name' key exists in the dictionary
            values.append(d['character'])
    return values
print(movies.shape[0])
for i in range(movies.shape[0]):
    arr = []
    titles.append(movies.iloc[i]['original_title'])
    overviews.append(movies.iloc[i]['overview'])
    res = json.loads(movies.iloc[i]['keywords'])
    keys = get_dict_values(res)
    describe = ""
    for k in range(len(keys)):
        describe = describe + " " +str(keys[k])
    res = json.loads(movies.iloc[i]['production_companies'])
    produce = get_dict_values(res)
    production = ""
    for k in range(len(produce)):
        production = production + " " +str(produce[k])
    res = json.loads(movies.iloc[i]['cast'])
    actor = get_dict_values(res)
    crew = ""
    for k in range(len(actor)):
        crew = crew + " " +str(actor[k])
    chars = get_charchter_values(res)
    character = ""
    for k in range(len(chars)):
        character = character + " " +str(chars[k])+" "
    characters.append(character)
    cast.append(crew)
    productions.append(production)
    keywords.append(describe)
    arr.append(production)
    arr.append(crew)
    arr.append(describe)
    arr.append(character)
    arr.append(movies.iloc[i]['original_title'])
    arr.append(movies.iloc[i]['overview'])
    for j in range(len(arr)):
      if isinstance(arr[j], list):
        for m in range(len(arr[j])):
          corpus2 = corpus2 + " " + str(arr[j][m])+" "
      else:
        corpus2 = corpus2 + str(arr[j]) + " "
    corpus2 = corpus2 + "."
print(corpus2)

4809
 It's the fictional tale of a wayward 9th grader, Ralph (Adam Butcher), who is secretly living on his own while his widowed, hospitalized mother remains immersed in a coma. Frequently in trouble with Father Fitzpatrick (Gordon Pinsent), the principal of his all-boys, Catholic school, Ralph is considered something of a joke among peers until he decides to pull off a miracle that could save his mother, i.e., winning the Boston Marathon.  Coached by a younger priest and former runner, Father Hibbert (Campbell Scott), whose cynicism has been lifted by the boy's pure hope, Ralph applies himself to his unlikely mission, fending off naysayers and getting help along a very challenging path from sundry allies and friends. . Maipo Film- og TV Produksjon  Jessica Chastain Colin Farrell Samantha Morton Nora McMenamy  love triangle love irish valet woman director 19th century  Miss Julie  John  Kristin  Little Miss Julie  Miss Julie Over the course of a midsummer night in Fermanagh in 1890, an

In [6]:
import gensim
from gensim.models import Word2Vec
sentences = corpus2.lower().split('.')
corpus3 = [sentence.strip().split() for sentence in sentences if sentence]
model = gensim.models.Word2Vec (corpus3, vector_size=150, window=5, min_count=2, workers=10)
model.train(corpus3,total_examples=len(corpus3),epochs=10)

(6960780, 7946540)

In [ ]:
for word in model.wv.index_to_key:
    print(word)

Streaming output truncated to the last 5000 lines.
cameo
mogwaza
ara
thatcher
velislav
(craig
claudiu
cities,
duchovny
fusco
apparatus
deems
razor-sharp
motley
lograsso
leach
neo-nazi
mokoena
intermittent
violence,
fana
indy
curmudgeonly
degrasse
witnessed
rosenfield
large-scale
lise
no,
closer,
naveen
tragedies
bardha
veil
chanadet
latina
lighter
cottage
gangale
zabryna
kyes
superman
downsizing
tamala
sloat
planet's
rickshaw
prieto
allure
beheading
patch
scenery
ivanovici
apergis
scarchilli
eerily
blows
standjofski
telefilm
regin
divoff
purwin
sunday
southwest
brookwell-mcnamara
hubley
unborn
blagrove
centers
prod
brodie
reshma
pokémon
malen
six-year-old
sivad
crooks
apparition
homemade
thai-hoa
carteret
tenuous
joung
tapes
esx
unexplainable
westerman
skagford
cornfoot
avco
leverman
zehra
guevara
violence?
pipe
humiliating
ribbons,
iko
chic
incompetent
mendez
birchwood
laina
buscapé
sheer
rendall
min
fransham
unwavering
buller
deus
cidade
melo
dangers
wakefield
dresses
trek
competitio

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_sentence_vector(sentence, model):
    words = sentence.lower().split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:  # If no word vectors found (e.g., for unknown words)
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

def get_combined_similarity(vector1, vector2, weights):
    """
    Calculate the combined similarity score using specified weights.
    """
    similarities = [cosine_similarity([v1], [v2])[0][0] for v1, v2 in zip(vector1, vector2)]
    weighted_similarity = sum(w * sim for w, sim in zip(weights, similarities))
    return weighted_similarity

# Input data
sentence1 = input("Enter the title: ")

# Find the matching sentence and append additional information
for i in range(len(titles)):
    if titles[i] == sentence1:
        sentence1_full = titles[i] + " " + keywords[i] + " " + productions[i] + " " + characters[i] + " " + cast[i]
        break
else:
    sentence1_full = sentence1  # If not found, use the input sentence as is

# Get vectors for different attributes of sentence1
vector1_title = get_sentence_vector(titles[i], model)
vector1_keywords = get_sentence_vector(keywords[i], model)
vector1_productions = get_sentence_vector(productions[i], model)
vector1_characters = get_sentence_vector(characters[i], model)
vector1_cast = get_sentence_vector(cast[i], model)

# Define weights for each attribute (titles, characters, cast, production, keywords)
weights = [0.9, 0.9, 0.6, 0.2, 0.1]

# Dictionary to store similarity votes
votes = {}

for k in range(len(titles)):
    if sentence1 == titles[k]:
        continue
    else:
        vector2_title = get_sentence_vector(titles[k], model)
        vector2_keywords = get_sentence_vector(keywords[k], model)
        vector2_productions = get_sentence_vector(productions[k], model)
        vector2_characters = get_sentence_vector(characters[k], model)
        vector2_cast = get_sentence_vector(cast[k], model)

        # Combine vectors into lists for similarity calculation
        vector1 = [vector1_title, vector1_characters, vector1_cast, vector1_productions, vector1_keywords]
        vector2 = [vector2_title, vector2_characters, vector2_cast, vector2_productions, vector2_keywords]

        # Calculate combined similarity
        similarity = get_combined_similarity(vector1, vector2, weights)
        votes[titles[k]] = similarity

# Sort the dictionary based on similarity scores
sorted_dict = sorted(votes.items(), key=lambda item: item[1], reverse=True)
print("the most recommend 5 movies for you are :")
print(sorted_dict[:5])


Enter the title: Harry Potter and the Half-Blood Prince
the most recommend 5 movies for you are :
[("Harry Potter and the Philosopher's Stone", 2.5707857012748723), ('Harry Potter and the Prisoner of Azkaban', 2.551558363437653), ('Harry Potter and the Chamber of Secrets', 2.5483324110507968), ('Harry Potter and the Order of the Phoenix', 2.503821223974228), ('Harry Potter and the Goblet of Fire', 2.434177941083908)]
